In [275]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [276]:
import pandas as pd
from timeit import default_timer as timer
import datetime
import numpy as np
from utils.config import PATH_SCENARIOS_REDUCED

In [277]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [278]:
t = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [279]:
node_df = pd.DataFrame(index=t.index)
node_df['node'] = 0

In [280]:
times = t.index.get_level_values('time').unique()
scenario_ids = t.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [[] for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else [list(scenario_ids)]

    found_ids = []

    values = t.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group

        for prev_group in prev_groups:
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i].append(group)
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
            nodes_counter += 1

In [282]:
node_df

node
time     scenarios      
00:00:00 0             0
         1             0
         2             0
         3             0
         4             0
...                  ...
20:00:00 27           80
         28           81
         29           82
         30           83
         31           84

[192 rows x 1 columns]